# Ingest population using water safelly overall

In [0]:
display(dbutils.fs.mounts())

mountPoint,source,encryptionType
/databricks-datasets,databricks-datasets,
/Volumes,UnityCatalogVolumes,
/mnt/waterprojectdl-raw,abfss://raw@waterprojectdl.dfs.core.windows.net/,
/databricks/mlflow-tracking,databricks/mlflow-tracking,
/databricks-results,databricks-results,
/mnt/waterprojectdl/raw,abfss://raw@waterprojectdl.dfs.core.windows.net/,
/databricks/mlflow-registry,databricks/mlflow-registry,
/Volume,DbfsReserved,
/volumes,DbfsReserved,
/,DatabricksRoot,


In [0]:
%fs 
ls /mnt/waterprojectdl/raw

path,name,size,modificationTime
dbfs:/mnt/waterprojectdl/raw/national_improved.csv,national_improved.csv,7464,1725728220000
dbfs:/mnt/waterprojectdl/raw/population_using_water_safely_overall.csv,population_using_water_safely_overall.csv,8729,1725723560000
dbfs:/mnt/waterprojectdl/raw/rural_improved.csv,rural_improved.csv,7390,1725728220000
dbfs:/mnt/waterprojectdl/raw/urban_imporved.csv,urban_imporved.csv,7394,1725728220000


In [0]:
overall = spark.read.csv("dbfs:/mnt/waterprojectdl/raw/population_using_water_safely_overall.csv", header=True)

In [0]:
display(overall)

SDG,SDG target,SDG indicator,Indicator Code,Indicator name,Geographical area code,Geographical area name,Year,Value,Time detail,Source,Footnote,Type of data,Units,Age group,Bounds,Frequency,Level/Status,Location,Type of reporting,Sex,SDG 6 Data portal level
null,null,6.1.1,null,"Drinking water,Safely managed service",null,Morocco,2000,41.516566077849,null,"WHO, UNICEF",null,null,null,null,null,null,null,National,null,null,6.1.1 Proportion of population using safely managed drinking water services > Safely managed service > Overall
null,null,6.1.1,null,"Drinking water,Safely managed service",null,Morocco,2001,41.955470588583,null,"WHO, UNICEF",null,null,null,null,null,null,null,National,null,null,6.1.1 Proportion of population using safely managed drinking water services > Safely managed service > Overall
null,null,6.1.1,null,"Drinking water,Safely managed service",null,Morocco,2002,42.393078989123,null,"WHO, UNICEF",null,null,null,null,null,null,null,National,null,null,6.1.1 Proportion of population using safely managed drinking water services > Safely managed service > Overall
null,null,6.1.1,null,"Drinking water,Safely managed service",null,Morocco,2003,42.828273466588,null,"WHO, UNICEF",null,null,null,null,null,null,null,National,null,null,6.1.1 Proportion of population using safely managed drinking water services > Safely managed service > Overall
null,null,6.1.1,null,"Drinking water,Safely managed service",null,Morocco,2004,43.261617365472,null,"WHO, UNICEF",null,null,null,null,null,null,null,National,null,null,6.1.1 Proportion of population using safely managed drinking water services > Safely managed service > Overall
null,null,6.1.1,null,"Drinking water,Safely managed service",null,Morocco,2005,43.805396222483,null,"WHO, UNICEF",null,null,null,null,null,null,null,National,null,null,6.1.1 Proportion of population using safely managed drinking water services > Safely managed service > Overall
null,null,6.1.1,null,"Drinking water,Safely managed service",null,Morocco,2006,44.369282773552,null,"WHO, UNICEF",null,null,null,null,null,null,null,National,null,null,6.1.1 Proportion of population using safely managed drinking water services > Safely managed service > Overall
null,null,6.1.1,null,"Drinking water,Safely managed service",null,Morocco,2007,44.929388561225,null,"WHO, UNICEF",null,null,null,null,null,null,null,National,null,null,6.1.1 Proportion of population using safely managed drinking water services > Safely managed service > Overall
null,null,6.1.1,null,"Drinking water,Safely managed service",null,Morocco,2008,47.739590836905,null,"WHO, UNICEF",null,null,null,null,null,null,null,National,null,null,6.1.1 Proportion of population using safely managed drinking water services > Safely managed service > Overall
null,null,6.1.1,null,"Drinking water,Safely managed service",null,Morocco,2009,50.602361224205,null,"WHO, UNICEF",null,null,null,null,null,null,null,National,null,null,6.1.1 Proportion of population using safely managed drinking water services > Safely managed service > Overall


In [0]:
from pyspark.sql.functions import col 
from pyspark.sql.types import DoubleType, IntegerType

overall_num = overall.withColumn("Value", col("Value").cast(DoubleType())) \
                 .withColumn("Year", col("Year").cast(IntegerType()))

In [0]:
print("Processed Schema:")
overall_num.printSchema()

Processed Schema:
root
 |-- SDG: string (nullable = true)
 |-- SDG target: string (nullable = true)
 |-- SDG indicator: string (nullable = true)
 |-- Indicator Code: string (nullable = true)
 |-- Indicator name: string (nullable = true)
 |-- Geographical area code: string (nullable = true)
 |-- Geographical area name: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Value: double (nullable = true)
 |-- Time detail: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- Footnote: string (nullable = true)
 |-- Type of data: string (nullable = true)
 |-- Units: string (nullable = true)
 |-- Age group: string (nullable = true)
 |-- Bounds: string (nullable = true)
 |-- Frequency: string (nullable = true)
 |-- Level/Status: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Type of reporting: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- SDG 6 Data portal level: string (nullable = true)



In [0]:
display(overall_num.filter(col("Year").isNotNull() & col("Value").isNotNull())
                   .select("Year", "Value")
                   .orderBy("Year"))

Year,Value
2000,41.516566077849
2001,41.955470588583
2002,42.393078989123
2003,42.828273466588
2004,43.261617365472
2005,43.805396222483
2006,44.369282773552
2007,44.929388561225
2008,47.739590836905
2009,50.602361224205


Databricks visualization. Run in Databricks to view.